In [1]:
#import libraries, compatible for 2 or 3
try:
    import urllib2 as ulib
except ImportError:
    import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
api_endpoint = 'https://fb55.carto.com:443/api/v2/sql?q='

In [3]:
#define function to send SQL query to Carto
def queryCartoDB(query, formatting = 'CSV', source = api_endpoint):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

In [4]:
#see what the table looks like
query = '''SELECT * FROM fb55.citibike; '''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


#### Task 1

In [5]:
#sort data by start_station_id, trip duration
query = '''SELECT * FROM fb55.citibike
ORDER BY start_station_id, tripduration;'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9059,-73.993929,107,2015-02-03 10:10:00+00,2015-02-03 10:12:00+00,72,8998,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19039,Subscriber,1980.0,2
1,NaN,W 52 St & 11 Ave,NaN,40772,-73.993929,189,2015-02-07 10:40:00+00,2015-02-07 10:43:00+00,72,40759,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,18131,Subscriber,1959.0,2
2,NaN,W 52 St & 11 Ave,NaN,40770,-73.993929,229,2015-02-07 10:40:00+00,2015-02-07 10:44:00+00,72,40757,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19729,Subscriber,1995.0,1
3,NaN,W 52 St & 11 Ave,NaN,22002,-73.993929,248,2015-02-05 06:19:00+00,2015-02-05 06:24:00+00,72,21948,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,16879,Subscriber,1970.0,1
4,NaN,W 52 St & 11 Ave,NaN,43107,-73.993929,252,2015-02-07 15:43:00+00,2015-02-07 15:47:00+00,72,43099,40.767272,515,W 43 St & 10 Ave,40.760094,-73.994618,15277,Subscriber,1983.0,1


In [6]:
#sort data by start_station_id, trip duration only checking trips with duration <= 3 hours (10,800 seconds)
query = '''SELECT * FROM fb55.citibike
WHERE tripduration <= 10800
ORDER BY start_station_id, tripduration;'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9059,-73.993929,107,2015-02-03 10:10:00+00,2015-02-03 10:12:00+00,72,8998,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19039,Subscriber,1980.0,2
1,NaN,W 52 St & 11 Ave,NaN,40772,-73.993929,189,2015-02-07 10:40:00+00,2015-02-07 10:43:00+00,72,40759,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,18131,Subscriber,1959.0,2
2,NaN,W 52 St & 11 Ave,NaN,40770,-73.993929,229,2015-02-07 10:40:00+00,2015-02-07 10:44:00+00,72,40757,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19729,Subscriber,1995.0,1
3,NaN,W 52 St & 11 Ave,NaN,22002,-73.993929,248,2015-02-05 06:19:00+00,2015-02-05 06:24:00+00,72,21948,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,16879,Subscriber,1970.0,1
4,NaN,W 52 St & 11 Ave,NaN,43107,-73.993929,252,2015-02-07 15:43:00+00,2015-02-07 15:47:00+00,72,43099,40.767272,515,W 43 St & 10 Ave,40.760094,-73.994618,15277,Subscriber,1983.0,1


In [7]:
#Only show the top/last 10 records (aka head and tail in SQL) 
query = '''SELECT * FROM fb55.citibike
LIMIT 10'''
table = get_data(query)
table.head(20)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0
5,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
6,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
7,NaN,E 39 St & 3 Ave,NaN,8327,-73.976049,441,2015-02-03 08:02:00+00,2015-02-03 08:10:00+00,167,8263,40.748901,526,E 33 St & 5 Ave,40.747659,-73.984907,21518,Subscriber,1975.0,1
8,NaN,10 Ave & W 28 St,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,9375,40.750664,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2
9,NaN,E 2 St & 2 Ave,NaN,10472,-73.990697,420,2015-02-03 16:45:00+00,2015-02-03 16:52:00+00,403,10411,40.725029,297,E 15 St & 3 Ave,40.734232,-73.986923,19257,Subscriber,1984.0,1


In [8]:
#List all unique start_station_id values 
query = '''SELECT start_station_id FROM fb55.citibike
GROUP BY start_station_id'''
table = get_data(query)
table

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN
5,144,NaN
6,266,NaN
7,263,NaN
8,216,NaN
9,328,NaN


In [9]:
#Count the number of trips (aka wc -l in SQL) 
query = '''SELECT COUNT(*) AS total_trips FROM fb55.citibike'''
table = get_data(query)
table

,total_trips,Unnamed: 1
0,46200,NaN


In [10]:
#Find the average/min/max trip duration
query = '''SELECT AVG(tripduration) AS avg_length, MIN(tripduration) AS min_trip, MAX(tripduration) AS max_trip FROM fb55.citibike'''
table = get_data(query)
table

,avg_length,min_trip,max_trip
0,675.865823,60,43016


#### Task Two

In [11]:
#Selecting trips started on Feb-02-2015 only 
query = '''SELECT * FROM fb55.citibike
WHERE starttime::date = '2015-02-02'
'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


In [12]:
#Selecting trips started on the weekends
query = '''SELECT * FROM fb55.citibike
WHERE EXTRACT(isodow FROM starttime::date) IN (6,7)
'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


In [13]:
#What are average trip duration during weekends? 
query = '''SELECT AVG(tripduration) AS weekend_average FROM fb55.citibike
WHERE EXTRACT(isodow FROM starttime::date) IN (6,7)
'''
table = get_data(query)
table.head()

,weekend_average,Unnamed: 1
0,662.942181,NaN


In [14]:
#Can we do the same for weekday?
query = '''SELECT AVG(tripduration) AS weekday_average FROM fb55.citibike
WHERE EXTRACT(isodow FROM starttime::date) NOT IN (6,7)
'''
table = get_data(query)
table.head()

,weekday_average,Unnamed: 1
0,681.052292,NaN


#### Task Three

In [15]:
#Showing the list of start station locations Using GROUP BY 
query = '''SELECT start_station_id FROM fb55.citibike
GROUP BY start_station_id'''
table = get_data(query)
table

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN
5,144,NaN
6,266,NaN
7,263,NaN
8,216,NaN
9,328,NaN


In [16]:
#Showing the number of trips started per station for stations within 500 meters of times square
query = '''SELECT start_station_id, COUNT(*) AS trips_started 
FROM fb55.citibike 
WHERE ST_Distance(ST_SetSrid(ST_MakePoint(-73.9857, 40.7577),4326), ST_SetSrid(ST_MakePoint(start_station_longitude, start_station_latitude),4326)) < 500
GROUP BY start_station_id
ORDER BY trips_started DESC
'''
table = get_data(query)
table.head()

,start_station_id,trips_started
0,521,530
1,435,519
2,293,511
3,477,507
4,318,486


#### Task Four

In [17]:
#Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare!
query = '''SELECT start_station_id, AVG(tripduration) AS avg_trip 
FROM fb55.citibike 
WHERE ST_Distance(ST_SetSrid(ST_MakePoint(-73.9857, 40.7577),4326), ST_SetSrid(ST_MakePoint(start_station_longitude, start_station_latitude),4326)) < 500
AND EXTRACT(isodow FROM starttime::date) IN (6,7)
GROUP BY start_station_id
ORDER BY avg_trip DESC
LIMIT 1
'''
table = get_data(query)
table

,start_station_id,avg_trip
0,492,1524.375
